<a href="https://colab.research.google.com/github/john-a-dixon/forecasting-food-sales/blob/main/forecasting-food-sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _**Forecasting Food Sales**_

###### **by John Andrew Dixon**

---

## Loading Data

#### *Data Dictionary*

|**Variable Name**        |**Description**                                                                                    |
|-------------------------|---------------------------------------------------------------------------------------------------|
|Item_Identifier          |Unique Product ID                                                                                  |
|Item_Weight              | Weight of product                                                                                 |
|Item_Fat_Content         |Whether the product is low fat or regular                                                          |
|Item_Visibility          |The percentage of total display area of all products in a store allocated to the particular product|
|Item_Type                |The category to which the product belongs                                                          |
|Item_MRP                 |Maximum Retail Price (list price) of the product                                                   |
|Outlet_Identifier        |Unique store ID                                                                                    |
|Outlet_Establishment_Year|The year in which the store was established                                                        |
|Outlet_Size              |	The size of the store in terms of ground area covered                                             |
|Outlet_Location_Type     |The type of area in which the store is located                                                     |
|Outlet_Type              |Whether the outlet is a grocery store or some sort of supermarket                                  |
|Item_Outlet_Sales        |Sales of the product in the particular store. This is the target variable to be predicted          |

#### *Imports & Load*

In [97]:
# Import the Pandas module
import pandas as pd

# Load the data into a DataFrame
food_sales_df = pd.read_csv('../../Datasets/sales_predictions.csv')

# Verification
food_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [98]:
# Take a quick look at the data
food_sales_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## Data Cleaning

#### *Inspecting Rows & Columns*

In [99]:
# Output the DataFrame's shape which is the number of rows and columns
rows_columns = food_sales_df.shape
rows_columns

(8523, 12)

> * There are 8523 rows with 12 columns.
#### *Inspecting Column Datatypes*

In [100]:
# Output the DataFrame's columns' datatypes
food_sales_df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

> * Based on it's name and Data Dictionary description, the column `Outlet_Size` ought to be a float or int. However, upon inspecting the table's head above, I see that the entries are categorical, *not* numerical. Thus, it's current `object` type makes sense. 
> * All other column datatypes align with what is expected as laid out in the Data Dictionary above.

#### *Inspecting For Duplicates*

In [101]:
# Count the number of duplicates
food_sales_df.duplicated().sum()

0

> * Fortunately, there are no duplicates within the data.

#### *Inspecting & Correcting Categorical Value Inconsistencies*

In [102]:
# Loop through the columns and output the value counts for each column 
# with potentially categorical values:
for column in food_sales_df.columns:
    if food_sales_df[column].dtype == 'object':
        print('***************************************************')
        print(column.upper())
        print(food_sales_df[column].value_counts())
        print()

***************************************************
ITEM_IDENTIFIER
FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
         ..
FDY43     1
FDQ60     1
FDO33     1
DRF48     1
FDC23     1
Name: Item_Identifier, Length: 1559, dtype: int64

***************************************************
ITEM_FAT_CONTENT
Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

***************************************************
ITEM_TYPE
Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dty

> * The `Item_Fat_Content` column contains inconsistences with it's two expected labels:
>   1. The label `Low Fat` is also entered as `LF` and `low fat`
>   2. The label `Regular` is also entered as `reg`

Fix the inconsistencies of the `Item_Fat_Content` column:

In [103]:
# Create a replacement dictionary specifying the proper replacements
replacement_dict = {
    'LF': 'Low Fat',
    'low fat': 'Low Fat',
    'reg': 'Regular'
}

# Apply the replacement dictionary inpace
food_sales_df['Item_Fat_Content'].replace(replacement_dict, inplace=True)

# Verify
food_sales_df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

#### _Identifying Missing Values_

In [104]:
# Ouput the number of missing values for each column
missing_data_count = food_sales_df.isna().sum()
missing_data_count

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [105]:
missing_item_weight_perc = missing_data_count['Item_Weight'] / len(food_sales_df)
missing_item_weight_perc

0.1716531737651062

In [106]:
missing_outlet_size_perc = missing_data_count['Outlet_Size'] / len(food_sales_df)
missing_outlet_size_perc

0.2827642848762173

> * About 17.165% of entries have a missing value within the `Item_Weight` column.
> * About 28.276% of entries have a missing value within the `Outlet_Size` column.

#### _Handling Missing Values_

First, I'll handle the missing values within the `Item_Weight` column. Note that `Item_Identifiers` are repeated in the dataset, as shown below:

In [107]:
food_sales_df['Item_Identifier'].value_counts().head()

FDW13    10
FDG33    10
NCY18     9
FDD38     9
DRE49     9
Name: Item_Identifier, dtype: int64

A small sample of three `Item_Identifier`s above (FDW13, FDG33, NCY18) show that the values in their `Item_Weights` column are the same except for when their is a missing value (i.e. One entry with `Item_Identifier` FDG33 has the same `Item_Weight` as another entry with `Item_Identifier` FDG33):

In [108]:
item_identifier_filter = (food_sales_df['Item_Identifier'] == 'FDW13') | (food_sales_df['Item_Identifier'] == 'FDG33') | (food_sales_df['Item_Identifier'] == 'NCY18')
food_sales_df.loc[item_identifier_filter, :].sort_values(by=['Item_Identifier'])

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
231,FDG33,NaN,Regular,0.139561,Seafood,170.4764,OUT027,1985,Medium,Tier 3,Supermarket Type3,3435.5280
7726,FDG33,5.365,Regular,0.140214,Seafood,173.7764,OUT035,2004,Small,Tier 2,Supermarket Type1,4637.9628
7098,FDG33,5.365,Regular,0.140124,Seafood,172.0764,OUT013,1987,High,Tier 3,Supermarket Type1,2748.4224
7011,FDG33,5.365,Regular,0.140812,Seafood,173.5764,OUT018,2009,Medium,Tier 3,Supermarket Type2,2061.3168
1202,FDG33,5.365,Regular,0.140525,Seafood,172.7764,OUT045,2002,NaN,Tier 2,Supermarket Type1,2233.0932
1575,FDG33,NaN,Regular,0.245543,Seafood,172.2764,OUT019,1985,Small,Tier 1,Grocery Store,171.7764
6112,FDG33,5.365,Regular,0.141034,Seafood,173.6764,OUT017,2007,NaN,Tier 2,Supermarket Type1,858.8820
5164,FDG33,5.365,Regular,0.140240,Seafood,171.0764,OUT046,1997,Small,Tier 1,Supermarket Type1,3263.7516
2728,FDG33,5.365,Regular,0.234733,Seafood,173.4764,OUT010,1998,NaN,Tier 3,Grocery Store,171.7764
4747,FDG33,5.365,Regular,0.140458,Seafood,169.7764,OUT049,1999,Medium,Tier 1,Supermarket Type1,3263.7516


I'll use this fact to fill in the missing values for each corresponding item weight. First, I'll get the mean item weight per `Item_Identifier`:

In [109]:
# Calculate average prices of each item, excluding entries with NaN
avg_item_price = food_sales_df.groupby('Item_Identifier')['Item_Weight'].mean()
avg_item_price

Item_Identifier
DRA12    11.600
DRA24    19.350
DRA59     8.270
DRB01     7.390
DRB13     6.115
          ...  
NCZ30     6.590
NCZ41    19.850
NCZ42    10.500
NCZ53     9.600
NCZ54    14.650
Name: Item_Weight, Length: 1559, dtype: float64

Then, I'll go through the data and replace each missing value in the `Item_Weight` column with the mean item weight of its corresponding `Item_Identifier`: 

In [110]:
is_NaN_filter = pd.isna(food_sales_df['Item_Weight']) 

for identifier in avg_item_price.index:
    item_identifier_filter = food_sales_df['Item_Identifier'] == identifier
    item_weight_average = avg_item_price[identifier]
    food_sales_df.loc[is_NaN_filter & item_identifier_filter, 'Item_Weight'] = item_weight_average

Finally, I'll verify how well this filled in the missing values of the `Item_Weight` column:

In [111]:
food_sales_df['Item_Weight'].isna().sum()

4

There are still 4 entries out of 8523 with missing data on the `Item_Weight` column. I'll remove them since losing 4 entries out of 8523 presents a negligible impact:

In [118]:
food_sales_df.dropna(subset=['Item_Weight'], inplace=True)
food_sales_df['Item_Weight'].isna().sum()

0

Handling the missing values within the `Outlet_Size` column will be different because it is a categorical, not a numerical variable. First, I want to see how the missing values break down across the `Outlet_Identifier` column since these values correspond to unique stores:

In [121]:
food_sales_df.groupby('Outlet_Identifier')['Outlet_Size'].value_counts(dropna=False)

Outlet_Identifier  Outlet_Size
OUT010             NaN            555
OUT013             High           932
OUT017             NaN            926
OUT018             Medium         928
OUT019             Small          527
OUT027             Medium         932
OUT035             Small          930
OUT045             NaN            929
OUT046             Small          930
OUT049             Medium         930
Name: Outlet_Size, dtype: int64

Based on the output, it appears entries with an `Outlet_Identifier` of OUT010, OUT017, or OUT045 all have wholly missing sizes. That is, no entry with those identifiers have size data. This may mean that the data collectors simply did not discern any sizes for these specific outlets. There is a potential to discern a pattern within the sizes of other outlets that may help make a judgement on what size the missing outlets are. I chose the columns that may have the most impact on `Outlet_Size`.

In [113]:
food_sales_df.groupby(['Outlet_Type', 'Outlet_Identifier'], dropna=False)['Outlet_Size'].value_counts(dropna=False, ascending=True)

Outlet_Type        Outlet_Identifier  Outlet_Size
Grocery Store      OUT010             NaN            555
                   OUT019             Small          527
Supermarket Type1  OUT013             High           932
                   OUT017             NaN            926
                   OUT035             Small          930
                   OUT045             NaN            929
                   OUT046             Small          930
                   OUT049             Medium         930
Supermarket Type2  OUT018             Medium         928
Supermarket Type3  OUT027             Medium         932
Name: Outlet_Size, dtype: int64

In [114]:
food_sales_df.groupby(['Outlet_Location_Type', 'Outlet_Identifier'], dropna=False)['Outlet_Size'].value_counts(dropna=False, ascending=True)

Outlet_Location_Type  Outlet_Identifier  Outlet_Size
Tier 1                OUT019             Small          527
                      OUT046             Small          930
                      OUT049             Medium         930
Tier 2                OUT017             NaN            926
                      OUT035             Small          930
                      OUT045             NaN            929
Tier 3                OUT010             NaN            555
                      OUT013             High           932
                      OUT018             Medium         928
                      OUT027             Medium         932
Name: Outlet_Size, dtype: int64

In [115]:
food_sales_df.groupby(['Outlet_Establishment_Year','Outlet_Identifier'], dropna=False)['Outlet_Size'].value_counts(dropna=False, ascending=True)

Outlet_Establishment_Year  Outlet_Identifier  Outlet_Size
1985                       OUT019             Small          527
                           OUT027             Medium         932
1987                       OUT013             High           932
1997                       OUT046             Small          930
1998                       OUT010             NaN            555
1999                       OUT049             Medium         930
2002                       OUT045             NaN            929
2004                       OUT035             Small          930
2007                       OUT017             NaN            926
2009                       OUT018             Medium         928
Name: Outlet_Size, dtype: int64

In [116]:
food_sales_df.groupby(['Outlet_Type', 'Outlet_Location_Type','Outlet_Identifier'], dropna=False)['Outlet_Size'].value_counts(dropna=False, ascending=True)

Outlet_Type        Outlet_Location_Type  Outlet_Identifier  Outlet_Size
Grocery Store      Tier 1                OUT019             Small          527
                   Tier 3                OUT010             NaN            555
Supermarket Type1  Tier 1                OUT046             Small          930
                                         OUT049             Medium         930
                   Tier 2                OUT017             NaN            926
                                         OUT035             Small          930
                                         OUT045             NaN            929
                   Tier 3                OUT013             High           932
Supermarket Type2  Tier 3                OUT018             Medium         928
Supermarket Type3  Tier 3                OUT027             Medium         932
Name: Outlet_Size, dtype: int64

## Exploratory Visuals

## Explanatory Visuals